In [41]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline , make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# knn imputer
from sklearn.impute import KNNImputer , SimpleImputer
from sklearn.model_selection import GridSearchCV

In [42]:
df = pd.read_csv(r'C:\Users\Hanif ullah laptop\Desktop\Churn Prediction\Data\processed\preprocessed_data.csv')
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [44]:

df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})
X = df.drop('Churn', axis=1)
y = df['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# --- Define Column Lists ---
cat_cols = [
    'gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
    'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling',
    'PaymentMethod'
]
num_cols = ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges']
ordered_cols = ['Contract']


# --- Create Pipelines for each column type ---

# 1. Create a sequential pipeline for NUMERICAL columns
# This will (1st) impute NaNs, and (2nd) scale the result
num_pipeline = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', StandardScaler())
])

# 2. Create a sequential pipeline for CATEGORICAL columns
# This will (1st) fill NaNs with a constant (e.g., 'missing'), and (2nd) one-hot encode
cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # or strategy='constant', fill_value='missing'
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))
    # handle_unknown='ignore to avoid errors during transform if new categories appear
])

# 3. Create a sequential pipeline for ORDERED columns
ord_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot_ord', OneHotEncoder(categories=[['Month-to-month', 'One year', 'Two year']], drop='first'))
])


# --- Build the FINAL Preprocessor ---
# Now, the ColumnTransformer applies each *pipeline* to its set of columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, num_cols),        # Apply num_pipeline to num_cols
        ('cat', cat_pipeline, cat_cols),        # Apply cat_pipeline to cat_cols
        ('ord', ord_pipeline, ordered_cols)     # Apply ord_pipeline to ordered_cols
    ])

In [53]:
#  model 

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(C=100, solver='liblinear'))
])

model.fit (X_train, y_train) 
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8190205819730305
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.90      0.88      1036
           1       0.68      0.59      0.63       373

    accuracy                           0.82      1409
   macro avg       0.77      0.75      0.76      1409
weighted avg       0.81      0.82      0.81      1409



In [50]:
param_grid = {
    'classifier__C': [0.01, 0.1, 1, 10, 100],
    'classifier__solver': ['liblinear', 'lbfgs']
}


grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

,estimator,Pipeline(step...egression())])
,param_grid,"{'classifier__C': [0.01, 0.1, ...], 'classifier__solver': ['liblinear', 'lbfgs']}"
,scoring,'accuracy'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('cat', ...), ...]"


In [ ]:
print("Best Parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_z

Best Parameters: {'classifier__C': 100, 'classifier__solver': 'liblinear'}
